In [ ]:
# default_exp core

# OUScope

> The telescope control is provided through the `Telescope` class which provides state tracking and low level methods - forming a basic API layer. The higher level functions are implemented as separate functions construced with the Telescope class API.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
class Telescope:
    pass